In [ ]:
nvidia-smi

In [ ]:
from ultralytics import YOLO
import cv2
# Load a pre-trained YOLOv8 model
model = YOLO('./models/best.pt')
model.to('cuda')  # Use GPU if available
video_path = './Assignment Materials/Assignment Materials/15sec_input_720p.mp4'

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    results = model(frame, stream=True)
    for result in results:
        result = result
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            conf = box.conf[0]
            cls = box.cls[0]
            label = f"{model.names[int(cls)]} {conf:.2f}"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.imshow('YOLOv8 Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        print("Exiting video stream.")
        break